In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [6]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ayshwarya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ayshwarya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [14]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [17]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [18]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [19]:
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-19-2539c7ec338f>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [20]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [21]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)

print("model created")

C:\Users\ayshwarya\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
10/10 [==============================] - 0s 991us/step - loss: 2.2176 - accuracy: 0.0851
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1317 - accuracy: 0.1702
Epoch 3/200
10/10 [==============================] - 0s 950us/step - loss: 2.1096 - accuracy: 0.1915
Epoch 4/200
10/10 [==============================] - 0s 886us/step - loss: 2.0277 - accuracy: 0.2553
Epoch 5/200
10/10 [==============================] - 0s 997us/step - loss: 1.8259 - accuracy: 0.4681
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.8722 - accuracy: 0.4043
Epoch 7/200
10/10 [==============================] - 0s 595us/step - loss: 1.6628 - accuracy: 0.5106
Epoch 8/200
10/10 [==============================] - 0s 884us/step - loss: 1.4697 - accuracy: 0.6170
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 1.3670 - accuracy: 0.6170
Epoch 10/200
10/10 [==============================] - 0s 817us/step - loss: 1.3348 - accuracy: 0.

10/10 [==============================] - 0s 886us/step - loss: 0.0627 - accuracy: 1.0000
Epoch 82/200
10/10 [==============================] - 0s 664us/step - loss: 0.0409 - accuracy: 1.0000
Epoch 83/200
10/10 [==============================] - 0s 805us/step - loss: 0.0971 - accuracy: 0.9362
Epoch 84/200
10/10 [==============================] - 0s 629us/step - loss: 0.0706 - accuracy: 0.9787
Epoch 85/200
10/10 [==============================] - 0s 693us/step - loss: 0.0347 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 664us/step - loss: 0.0373 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 587us/step - loss: 0.0400 - accuracy: 0.9787
Epoch 88/200
10/10 [==============================] - 0s 664us/step - loss: 0.1334 - accuracy: 0.9149
Epoch 89/200
10/10 [==============================] - 0s 578us/step - loss: 0.0526 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 554us/step - loss: 0.0330 - accuracy: 

10/10 [==============================] - 0s 554us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 161/200
10/10 [==============================] - 0s 768us/step - loss: 0.0666 - accuracy: 0.9787
Epoch 162/200
10/10 [==============================] - 0s 655us/step - loss: 0.0355 - accuracy: 1.0000
Epoch 163/200
10/10 [==============================] - 0s 769us/step - loss: 0.0976 - accuracy: 0.9787
Epoch 164/200
10/10 [==============================] - 0s 665us/step - loss: 0.0735 - accuracy: 0.9787
Epoch 165/200
10/10 [==============================] - 0s 691us/step - loss: 0.0068 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 754us/step - loss: 0.0129 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 554us/step - loss: 0.0100 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 726us/step - loss: 0.0418 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 664us/step - loss: 0.0116 - a